In [1]:
#  INSTALACIÓN DE DEPENDENCIAS

!pip install segment-anything opencv-python pytesseract transformers easyocr tqdm pandas --quiet
!pip install git+https://github.com/facebookresearch/segment-anything.git --quiet
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O sam_vit_b.pth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
--2026-02-23 23:32:22--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.167.112.129, 3.167.112.66, 3.167.112.53, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.167.112.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b.pth’

sam_vit_b.pth       100%[===================>] 357.67M   272MB/s    in 1.3s    

2026-02-23 23:32:24 (272 MB/s) - ‘sam_vit_b.pth’ saved [375042383/375042383]



In [2]:
#MONTAR GOOGLE DRIVE Y EXTRAER ZIP

from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/Copia de 20260121 imagenes Cashcollection1.zip"
extract_dir = "/content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/dataset_bimbonet"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)

print("ZIP extraído en:", extract_dir)

IMAGE_DIR = os.path.join(extract_dir, "20260121 imagenes Cashcollection")
print("Carpeta de imágenes:", IMAGE_DIR)

print("Ejemplo de archivos:", os.listdir(IMAGE_DIR)[:5])

Mounted at /content/drive
ZIP extraído en: /content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/dataset_bimbonet
Carpeta de imágenes: /content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/dataset_bimbonet/20260121 imagenes Cashcollection
Ejemplo de archivos: ['387.png', '388.png', '39.png', '391.png', '390.png']


In [3]:
#IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN

import cv2
import torch
import numpy as np
import pytesseract
import re
import pandas as pd
from tqdm import tqdm

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import easyocr

device = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", device)

DEVICE: cuda


In [4]:
#CARGA DE MODELOS (SAM, DINO, EASYOCR)

# SAM
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth").to(device)
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=4,
    pred_iou_thresh=0.88,
    stability_score_thresh=0.90,
    min_mask_region_area=300
)

# DINO
processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")
dino = AutoModelForZeroShotObjectDetection.from_pretrained(
    "IDEA-Research/grounding-dino-base"
).to(device)

# EasyOCR
reader = easyocr.Reader(['es', 'en'], gpu=(device=="cuda"))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

The image processor of type `GroundingDinoImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/933M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/1206 [00:00<?, ?it/s]

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [5]:
#FUNCIÓN DE REGEX PARA EXTRAER TEXTO

def extract_col_strict(text):
    text = text.upper()
    pattern = r"C[O0]L\s*([123])[\.\,\s]*(\d{5})\s+(\d{5})"
    matches = re.findall(pattern, text)
    return [f"COL {m[0]}. {m[1]} {m[2]}" for m in matches]



In [6]:
#FUNCIÓN DE EXTRACCIÓN CON SAM

def sam_extract(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_small = cv2.resize(img_rgb, (1024, 1024))

    masks = mask_generator.generate(img_small)
    found = []

    for m in masks:
        mask = m["segmentation"].astype(np.uint8) * 255
        x, y, w, h = cv2.boundingRect(mask)
        crop = img_small[y:y+h, x:x+w]

        if crop.size == 0:
            continue

        ocr = pytesseract.image_to_string(crop)
        cleaned = extract_col_strict(ocr)
        if cleaned:
            found.extend(cleaned)

    return sorted(list(set(found)))

In [7]:
#FUNCIÓN DE FALLBACK CON DINO

def dino_fallback(img):
    if img is None or img.size == 0:
        return []

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    inputs = processor(images=img_rgb, text="printed text with numbers", return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs, inputs.input_ids,
        threshold=0.22, text_threshold=0.22,
        target_sizes=[img.shape[:2]]
    )[0]

    found = []
    h, w = img.shape[:2]

    for box in results["boxes"]:
        x1, y1, x2, y2 = map(int, box.tolist())

        x1 = max(0, min(x1, w - 1))
        x2 = max(0, min(x2, w - 1))
        y1 = max(0, min(y1, h - 1))
        y2 = max(0, min(y2, h - 1))

        if x2 <= x1 or y2 <= y1:
            continue

        crop = img[y1:y2, x1:x2]

        if crop is None or crop.size == 0:
            continue

        try:
            ocr_list = reader.readtext(crop, detail=0)
        except:
            continue

        text = " ".join(ocr_list)
        cleaned = extract_col_strict(text)

        if cleaned:
            found.extend(cleaned)

    return sorted(list(set(found)))

In [8]:
#PIPELINE HÍBRIDO SAM → DINO

def extract_col_hybrid(img_path):
    img = cv2.imread(img_path)

    # Primero intentar con SAM
    sam_result = sam_extract(img)
    if sam_result:
        return sam_result

    # Si SAM falla → usar DINO
    return dino_fallback(img)

In [9]:
#PROCESAR SOLO 50 IMÁGENES + BARRA DE PROGRESO

all_images = sorted([
    f for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith((".jpg", ".jpeg", ".png", ".jfif"))
])

subset_images = all_images[:50]

rows = []

for img_name in tqdm(subset_images, desc="Procesando imágenes"):
    path = os.path.join(IMAGE_DIR, img_name)
    detected = extract_col_hybrid(path)

    col1 = next((x for x in detected if x.startswith("COL 1")), "")
    col2 = next((x for x in detected if x.startswith("COL 2")), "")
    col3 = next((x for x in detected if x.startswith("COL 3")), "")

    rows.append({"imagen": img_name, "col1": col1, "col2": col2, "col3": col3})


Procesando imágenes: 100%|██████████| 50/50 [06:22<00:00,  7.66s/it]


In [11]:
#TABLA FINAL DE RESULTADOS

df_results = pd.DataFrame(rows)
df_results.head()

,imagen,col1,col2,col3
0,1.jpeg,COL 1. 15355 50899,COL 2. 01146 94344,COL 3. 01816 44947
1,10.jpeg,,,
2,100.jpg,,COL 2. 30669 56415,COL 3. 69800 86149
3,101.png,COL 1. 96476 95643,COL 2. 00069 36425,COL 3. 60880 89364
4,102.png,,COL 2. 51144 64548,COL 3. 91826 13568


In [ ]:
import json
import nbformat
import requests
from google.colab import _message

# Obtener el contenido del notebook actual
nb_json = _message.blocking_request('get_ipynb')['ipynb']
nb = nbformat.from_dict(nb_json)

# Limpiar metadata
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

for cell in nb["cells"]:
    if "metadata" in cell:
        cell["metadata"] = {}

# Guardar archivo limpio
output_name = "notebook_limpio.ipynb"
with open(output_name, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

output_name